In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random
import scipy.stats
import plotly
import geopandas as gpd
from IPython.display import Markdown
from dash import Dash, dcc, html, Input, Output
import plotly.express as px
%matplotlib inline

In [ ]:
#table1 = pd.read_csv(r'C:\Users\aleks\OneDrive\Desktop\DataVisualization\RV_O_010_SK\RV_O_010_L_OK_SK.csv', sep=';')
#table2 = pd.read_csv(r'C:\Users\aleks\OneDrive\Desktop\DataVisualization\RV_O_040_SK\RV_O_040_L_OK_SK.csv', sep=';')
#table3 = pd.read_csv(r'C:\Users\aleks\OneDrive\Desktop\DataVisualization\RV_O_047_SK\RV_O_047_L_OK_SK.csv', sep=';')
table4 = pd.read_csv(r'C:\Users\aleks\OneDrive\Desktop\DataVisualization\RV_O_067_SK\RV_O_067_L_OK_SK.csv', sep=';')
table4_pom = pd.read_csv(r'C:\Users\aleks\OneDrive\Desktop\DataVisualization\RV_O_067_SK\RV_O_067_R_KR_SK.csv', sep=';')

In [ ]:
table4_pom.head(5)

In [ ]:
display(sum(table4_pom['abs.']))

In [ ]:
vyssie_zoznam = [x for x in table4.iloc[:, 5].unique() if 'vysokoškolské' in x or "vyššie" in x]
stredne_zoznam = [x for x in table4.iloc[:, 5].unique() if "stredné" in x]
zakladne_zoznam = [x for x in table4.iloc[:, 5].unique() if "základné" in x]
neziskane_zoznam = ['bez školského vzdelania – osoby vo veku 15 rokov a viac'] 
ostatne_zoznam = [x for x in table4.iloc[:, 5].unique() if x not in vyssie_zoznam and x not in stredne_zoznam 
                  and x not in zakladne_zoznam and x not in neziskane_zoznam]

In [ ]:
vzdelanie_dict = {'vyššie': vyssie_zoznam,
                  'stredné': stredne_zoznam,
                  'základné': zakladne_zoznam,
                  'neziskané': neziskane_zoznam,
                  'ostatné': ostatne_zoznam}

def det_education(vz):
    for ed_type in vzdelanie_dict.keys():
        if vz in vzdelanie_dict.get(ed_type):
            return ed_type

In [ ]:
table4_pom['Druh vzdelania'] = table4_pom['Vzdelanie'].apply(det_education)

In [ ]:
education_series = table4_pom.groupby('Druh vzdelania')['abs.'].sum()

In [ ]:
counts_df = pd.DataFrame({'Druh vzdelania': education_series.index,
                          'Počet ľudí': education_series.values})

In [ ]:
counts_df

In [ ]:
figure, axes = plt.subplots(figsize=(10, 10))
plt.bar(counts_df['Druh vzdelania'], counts_df['Počet ľudí'])
axes.semilogy()
plt.title('The number of people by achieved education')
plt.show()

Chcem preskúmať súvis medzi druhom zamestnania/pracovnou oblasťou (budú utriedené podľa ISCO klasifikácie, ktorá je zverejnená na ich webovej stránke (9 skupín)) a ziskaným vzdelaním. Sú to dve kategoriálne premenné, takže viem vyrobiť z toho "Parallel categories" graf.

1) Roztriediť zamestnania a dostať deväť skupín
2) Nakresliť graf

In [ ]:
groups = {'Managers': ['Riadiaci pracovníci (manažéri)  administratívnych, podporných a obchodných  činností',
                'Riadiaci pracovníci (manažéri) vo výrobe a v špecializovaných službách',
                'Riadiaci pracovníci (manažéri) v hotelových, reštauračných, obchodných a v ostatných službách',
                'Zákonodarcovia, ústavní činitelia, vysokí štátni úradníci a najvyšší predstavitelia podnikov a organizácií'],
         'Professionals': ['Špecialisti v zdravotníctve', 'Špecialisti v oblasti práva, sociálnych vecí a kultúry',
                'Špecialisti administratívnych, podporných a obchodných činností', 
                'Špecialisti v oblasti vedy a techniky', 'Špecialisti v oblasti informačných a komunikačných technológií',
                'Učitelia a odborní pedagogickí pracovníci'],
         'Technicians and Associate Professionals': ['Technici a odborní pracovníci v oblasti vedy a techniky', 
                'Technici v oblasti informačných a komunikačných technológií', 
                'Odborní pracovníci administratívnych, podporných a obchodných činností', 'Odborní pracovníci v zdravotníctve',
                'Odborní pracovníci v oblasti práva, sociálnych vecí a kultúry a podobní pracovníci'],
         'Clerical Support Workers': ['Administratívni pracovníci na záznam číselných a skladových údajov',
                'Všeobecní administratívni pracovníci a zapisovatelia', 'Administratívni pracovníci v zákazníckych službách',
                'Ostatní pomocní administratívni pracovníci'],
         'Service and Sales Workers': ['Pracovníci v osobných službách', 'Predavači', 
                'Pracovníci v oblasti osobnej starostlivosti', 'Pracovníci verejnej ochrany a bezpečnostných služieb'],
         'Skilled Agricultural, Forestry and Fishery Workers': ['Kvalifikovaní pracovníci v poľnohospodárstve (trhovo orientovaní)',
                'Kvalifikovaní pracovníci v lesníctve, rybárstve a poľovníctve (trhovo orientovaní)',
                'Farmári, rybári, poľovníci a zberači úrody (samozásobovatelia)'],
         'Craft and Related Trades Workers': ['Kvalifikovaní stavební robotníci a remeselníci okrem elektrikárov',
                'Kvalifikovaní robotníci v hutníctve, strojárstve a podobní robotníci', 'Umeleckí a ruční remeselníci a tlačiari',
                'Elektrikári a elektronici', 'Spracovatelia a výrobcovia potravinárskych výrobkov, výrobkov z dreva a odevov'],
         'Plant and Machine Operators, and Assemblers': ['Operátori stacionárnych strojov a zariadení', 'Montážni robotníci',
                'Vodiči a obsluha pojazdných strojných zariadení'],
         'Elementary Occupations': ['Upratovači a pomocníci', 'Pomocní pracovníci v poľnohospodárstve, lesníctve a rybárstve',
                'Pomocní pracovníci v ťažbe, stavebníctve, výrobe a doprave', 'Pomocní pracovníci pri príprave jedla',
                'Pracovníci pri likvidácii odpadu a ostatní nekvalifikovaní pracovníci',
                'Pouliční predavači a pomocní pracovníci v podobných službách'],
         'Armed Forces Occupations': ['Dôstojníci ozbrojených síl', 'Poddôstojníci ozbrojených síl', 'Ostatné ozbrojené sily']}

In [ ]:
economic_sectors = {'Undefined': ['nezistené'],
                     'Other activities': ['Ostatné činnosti'],
                     'Primary': ['Poľnohospodárstvo, lesníctvo a rybolov', 'Ťažba a dobývanie'],
                     'Secondary': ['Stavebníctvo', 'Priemyselná výroba'],
                     'Tertiary': ['Veľkoobchod a maloobchod; oprava motorových vozidiel a motocyklov',
                            'Doprava a skladovanie', 'Ubytovacie a stravovacie služby', 'Informácie a komunikácia',
                            'Činnosti v oblasti nehnuteľností', 'Odborné, vedecké a technické činnosti',
                            'Administratívne a podporné služby', 'Vzdelávanie', 'Umenie, zábava a rekreácia',
                            'Činnosti extrateritoriálnych organizácií a združení', 'Zdravotníctvo a sociálna pomoc',
                            'Dodávka elektriny, plynu, pary a studeného vzduchu', 'Finančné a poisťovacie činnosti',
                            'Dodávka vody; čistenie a odvod odpadových vôd, odpady a služby odstraňovania odpadov',
                            'Verejná správa a obrana; povinné sociálne zabezpečenie',
                            'Činnosti domácností ako zamestnávateľov; nediferencované činnosti v domácnostiach produkujúce tovary a služby na vlastné použitie']}

In [ ]:
table_oc_groups = []
group_names = ['Managers', 'Professionals', 'Technicians and Associate Professionals', 'Clerical Support Workers', 
              'Service and Sales Workers', 'Skilled Agricultural, Forestry and Fishery Workers', 'Armed Forces Occupations',
              'Craft and Related Trades Workers', 'Plant and Machine Operators, and Assemblers', 'Elementary Occupations']

def group_func(zamestnanie):
    for name in group_names:
        if zamestnanie in groups.get(name):
            return name
        
    return 'Undefined'

table4_pom['Druh zamestnania (ISCO - triedy)'] = table4_pom['Zamestnanie (ISCO - triedy)'].apply(group_func)

In [ ]:
sector_names = ['Undefined', 'Other activities', 'Primary', 'Secondary', 'Tertiary']

def sector_func(odvetie):
    for sector in sector_names:
        if odvetie in economic_sectors.get(sector):
            return sector

table4_pom['Ekonomický sektor (NACE - sekcie)'] = table4_pom['Odvetvie ekonomickej činnosti​ (NACE - sekcie)'].apply(sector_func)

In [ ]:
education_types = []
occupation_groups = []
regions = []
sectors = []

for index, row in table4_pom.iterrows():
    for ed_type in vzdelanie_dict.keys():
        if row['Vzdelanie'] in vzdelanie_dict.get(ed_type):
            item = ed_type
            break
        
    for i in range(row['abs.']):
        education_types.append(item)
        occupation_groups.append(row['Druh zamestnania (ISCO - triedy)'])
        regions.append(row['Názov kraja'])
        sectors.append(row['Ekonomický sektor (NACE - sekcie)'])

In [ ]:
table_for_parallel_cat = pd.DataFrame(data={'Druh vzdelania': education_types,
                                           'Druh zamestnania': occupation_groups,
                                           'Názov kraja': regions,
                                           'Ekonomický sektor': sectors})

In [ ]:
display(table_for_parallel_cat)

In [ ]:
'''
for_parallel_cat = (table_for_parallel_cat.loc[:,['Druh vzdelania','Druh zamestnania']])
fig = px.parallel_categories(for_parallel_cat, width=800)
fig.update_traces(dimensions=[{"categoryorder": "array"}, {}])
fig.update_layout(font_size=10)
fig.update_layout(margin={'l':100,'r':100})
fig.show()
'''

In [ ]:
'''
for_parallel_cat = (table_for_parallel_cat.loc[:,['Názov kraja','Druh vzdelania']])
fig = px.parallel_categories(for_parallel_cat, width=800)
fig.update_traces(dimensions=[{"categoryorder": "array"}, {}])
fig.update_layout(font_size=10)
fig.update_layout(margin={'l':100,'r':100})
fig.show()
'''

In [ ]:
display(table_for_parallel_cat.groupby('Ekonomický sektor').count())

In [ ]:
d = {'Other activities': 49446, 'Primary': 64378, 'Secondary': 620614, 'Tertiary': 1427411, 'Undefined': 392042}

def f(se):
    return d.get(se)

In [ ]:
table_for_parallel_cat['Súčet vnútri sektora'] = table_for_parallel_cat['Ekonomický sektor'].apply(f)

In [ ]:
display(table_for_parallel_cat.groupby(['Súčet vnútri sektora', 'Druh vzdelania']).count())

In [ ]:
grouped_table = table_for_parallel_cat.groupby(['Súčet vnútri sektora', 'Druh vzdelania']).count()

def groups_count(dv, svs):
    return grouped_table.loc[(svs, dv), :].values[0]

In [ ]:
table_for_parallel_cat['Súčet pre vzdelanie'] = table_for_parallel_cat.apply(lambda x: groups_count(x['Druh vzdelania'], x['Súčet vnútri sektora']), axis=1)

In [ ]:
table_for_parallel_cat['Percentage'] = table_for_parallel_cat['Súčet pre vzdelanie'] * 100 / table_for_parallel_cat['Súčet vnútri sektora']

In [ ]:
table_for_parallel_cat

In [ ]:
tmp_sectors = table_for_parallel_cat['Ekonomický sektor'].unique()
tmp_education = table_for_parallel_cat['Druh vzdelania'].unique()
# the first rectangles start at 0
starts = pd.Series([0] * tmp_sectors.shape[0])
# create plot
figure, axes = plt.subplots()

# iterate through income groups
for group in tmp_education:  
    # select data for this income group from both regions
    tmp_data = table_for_parallel_cat.query("`Druh vzdelania` == @group").drop_duplicates('Súčet pre vzdelanie')
    # plot
    rectangles = axes.barh(y=tmp_data['Ekonomický sektor'], width=tmp_data['Percentage'], left=starts, label=group)
    # add labels
    axes.bar_label(rectangles, label_type='center', fmt="%.0f%%")
    # move starts by the size of each rectangle
    starts += tmp_data['Percentage'].reset_index(drop=True) 

axes.set_title('The sixth graph')
axes.legend(bbox_to_anchor=(1, 1), loc=2)
# hide plot frame and x-axis ticks
axes.xaxis.set_visible(False)
axes.set_frame_on(False)
pass

In [ ]:
table_for_parallel_cat2 = table_for_parallel_cat.drop_duplicates(['Druh vzdelania', 'Ekonomický sektor', 'Súčet vnútri sektora'])
#display(table_for_parallel_cat2)

In [ ]:
figure, axes = plt.subplots(figsize=(8,5))
axes.pie(x=table_for_parallel_cat2.query('`Druh vzdelania` == "neziskané"')['Súčet pre vzdelanie'], labels=table_for_parallel_cat2.query('`Druh vzdelania` == "neziskané"')['Ekonomický sektor'], autopct="%.1f%%")
axes.set_title('People without education by economic sectors')
pass

In [ ]:
figure, axes = plt.subplots(figsize=(8,5))
axes.pie(x=table_for_parallel_cat2.query('`Druh vzdelania` == "vyššie"')['Súčet pre vzdelanie'], labels=table_for_parallel_cat2.query('`Druh vzdelania` == "vyššie"')['Ekonomický sektor'], autopct="%.1f%%")
axes.set_title('People with higher education by economic sectors')

In [ ]:
figure, axes = plt.subplots(figsize=(8,5))
axes.pie(x=table_for_parallel_cat2.query('`Druh vzdelania` == "základné"')['Súčet pre vzdelanie'], labels=table_for_parallel_cat2.query('`Druh vzdelania` == "základné"')['Ekonomický sektor'], autopct="%.1f%%")
axes.set_title('People with elementary education by economic sectors')

In [ ]:
table_for_parallel_cat

In [ ]:
# create a list of all functions
#all_functions = list(functions['function'].unique())

# create a new dash application app
app = Dash(__name__)

# Create layout of items in application
#   one html <div> item containing text as small headwers (H4), 
#   items for individual inputs and a graph at the bottom
# Currently we have two inputs:
#   an input field for entering title text
#   checkboxes for selecting functions 
# These elements have identifiers which will be used later in the code
app.layout = html.Div([
    html.Div(children=[
        html.Label('Vzdelanie: '),
        dcc.Dropdown(['Bez školského vzdelania', 'Základné vzdelanie', 'Stredné vzdelamie',
                     'Vyššie vzdelanie', 'Ostatné'], 'Základné vzdelanie', id='education-type')]),
    
    html.Div(children=[
        html.Label('Zafarbit podla ekonomického sektoru: '),
        dcc.RadioItems(['áno', 'nie'], 'nie', id='color-choice')
    ]),
    
    dcc.Graph(id='graph-content')
])

# @app.callback is a function decorator applied to function update_figure below.
# It defines that this function will be called to update the graph when the user makes a change.
#    Input will be the value entered to the input field with id graph-title and 
#      the list of functions selected in dcc.Checklist object with id 'selected-functions'.
#    Output will be the graph created by the function update_figure below, which will be used
#      to update dcc.Graph object with id 'graph-content'
@app.callback(
    Output('graph-content', 'figure'),
    [
        Input('education-type', 'value'),
        Input('color-choice', 'value')
    ]
)
def update_figure(selected_education, color_c):
    """ Function for ploting functions listed in list selected_functions
    with plot title given in title"""
    education_dict = {'Bez školského vzdelania': 'neziskané',
                      'Základné vzdelanie': 'základné',
                      'Stredné vzdelamie': 'stredné',
                      'Vyššie vzdelanie': 'vyššie',
                      'Ostatné vzdelanie': 'ostatné'}
    education = education_dict.get(selected_education)
    
    if color_c == 'áno':
        color_c = 'Ekonomický sektor'
    else:
        color_c = None
    
    # select a subset of functions table with just those functions in input list
    education_subset = table_for_parallel_cat.query('`Druh vzdelania` == @education')
    
    # create a plotly line plot using the smaller table in functions_subset
    figure = px.histogram(education_subset, x="Druh zamestnania", width=900, height=600, color=color_c)

    # add title to the plot
    figure.update_layout(title_text='Zamestnanie podla vzdelania')    

    return figure

# run the whole application
app.run_server(mode='inline')

In [ ]:
table_for_parallel_cat.memory_usage()

In [ ]:
table4.memory_usage()